## Agent Churn Prediction
## Data used for analysis is based on Integral Life Admin System

### The SQL query in below cells will retreive data based on database credentials provide. Right now these credentials have been left empty for security reasons. Please enter relevant details before running this notebook further.

In [1]:
# Import Packages
import numpy as np
import pandas as pd
import pyodbc

# Get Current time
import datetime 

# For Graphics
import matplotlib as matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# For splitting data into test and train subsets
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, GridSearchCV

# For logistic regression
from sklearn.linear_model import LogisticRegression

# For saving the model
from sklearn.externals import joblib 

# For Confusion Matrix
from sklearn import metrics

# For ROC Curve
from sklearn.metrics import roc_auc_score, roc_curve

# For Normalization
from sklearn.preprocessing import MinMaxScaler

# For Feature Selection
from sklearn.feature_selection import SelectKBest, chi2

# Settings to view all columns and rows
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

C:\Users\aaacharya\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


## Import Data

In [2]:
# Enter ip address and port number of the system where the database(20.4 regression) resides.
server = '10.0.3.172'
database = 'INT77DB_204' 
username = 'sisensedb_user'
password = 'Sisense12#$' 

In [3]:
# Add appropriate driver name
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

OperationalError: ('08001', '[08001] [Microsoft][ODBC SQL Server Driver][DBNETLIB]SQL Server does not exist or access denied. (17) (SQLDriverConnect); [08001] [Microsoft][ODBC SQL Server Driver][DBNETLIB]ConnectionOpen (Connect()). (53)')

In [ ]:
# Run Query
query = '''
select 
q1.agntnum	as 'Agent Number',
q1.dteapp	as 'Date_of_Appointment',
case
when
q1.dtetrm = '99999999'
then 
''
else
cast(q1.dtetrm as varchar)
end			as 'Date_of_Termination',
q1.agcls	as 'Commission Class',
q1.paymth	as 'Pay Method',
q1.payfrq	as 'Pay Frequency',
q2.agtype	as 'Agent Type',
q2.agntbr   as 'Agent Branch Code',
q2.replvl	as 'Agent Reporting Level',
case
when 
q3.cltdob = '99999999'
then 
''
else
cast (q3.cltdob as varchar)
end			as 'Agent_Date_of_Birth',
q3.cltsex	as 'Agent Gender',
isnull(q4.policies,0)  as 'Number_of_Policies',
isnull(q7.claims,0)  as 'Number_of_Claims',
isnull(q6.claim_amount,0)  as 'Total_Claim_Amount',
isnull(q5.payments,0)  as 'Total_Payment_Amount',
isnull(q8.commission,0)  as 'Total_Commission_Amount',
case
when
q1.dtetrm = 99999999
then 
0
else
1
end
as 'Churn'

from 
(select agntnum, dteapp, dtetrm, agcls, paymth, payfrq from vm1dta.aglfpf where agntcoy='2') q1
inner join 
(select agntnum, clntnum, agtype, agntbr, replvl from vm1dta.agntpf where agntcoy='2') q2
on q1.agntnum = q2.agntnum
inner join 
(select * from vm1dta.clntpf where validflag='1') q3
on q3.clntnum = q2.clntnum
left outer join
(select agntnum, count(distinct chdrnum) as policies from vm1dta.chdrpf where chdrcoy = '2' group by agntnum )q4
on q4.agntnum = q1.agntnum 
left outer join
(
select 
chdr.agntnum, sum(regp.payment) as payments
from 
(select distinct agntnum, chdrnum from vm1dta.chdrpf where chdrcoy = '2') chdr 
inner join
(select distinct chdrnum, sum(totamnt) as payment from vm1dta.regppf group by chdrnum) regp
on regp.chdrnum = chdr.chdrnum
group by chdr.agntnum
)q5
on q5.agntnum = q1.agntnum 
left outer join
(
select 
chdr.agntnum, sum(clmd.amount) as claim_amount
from 
(select distinct agntnum, chdrnum from vm1dta.chdrpf where chdrcoy = '2') chdr 
inner join
(select sum(actvalue) as amount, chdrnum from vm1dta.clmdpf group by chdrnum ) clmd
on clmd.chdrnum = chdr.chdrnum
group by chdr.agntnum
)q6
on q6.agntnum = q1.agntnum 
left outer join 
(
select 
chdr.agntnum, count(clmd.chdrnum) as claims
from 
(select distinct agntnum, chdrnum from vm1dta.chdrpf where chdrcoy = '2') chdr 
inner join
(select sum(actvalue) as amount, chdrnum from vm1dta.clmdpf group by chdrnum ) clmd
on clmd.chdrnum = chdr.chdrnum
group by chdr.agntnum
)q7
on q7.agntnum = q1.agntnum 
left outer join 
(select rldgacct, sum(acctamt) as commission from vm1dta.acmvpf where rdoccoy = '2' and SACSCODE='LA'  AND SACSTYP IN ('IC','RC','SG')
group by rldgacct)q8
on q8.rldgacct = q1.agntnum

order by q1.agntnum
'''

In [ ]:
# Read data
df_orig = pd.read_sql(query,cnxn)

# Close the cursor
cursor.close() 

In [ ]:
# Check data sample
df_orig.head()

## Data Analysis & Visualization

In [ ]:
# Check data shape
df_orig.shape

In [ ]:
# Check data attributes of the columns
df_orig.info()

In [ ]:
# View distribution of 'Churn' Instances
df_orig['Churn'].value_counts()

In [ ]:
# Check if there are any nulls
df_orig.isnull().sum()

In [ ]:
# Plot count values
df_orig.Churn.value_counts()
sns.countplot(x = 'Churn', data = df_orig, palette = 'hls')
plt.show()

## Data Treatment

In [ ]:
# Create a copy of dataframe
df_work = df_orig.copy()

In [ ]:
# Separate features into categorical, numerical and category
cat_vars = ['Commission Class','Pay Method','Pay Frequency','Agent Type','Agent Branch Code',
            'Agent Reporting Level','Agent Gender']

num_vars = ['Number_of_Policies', 'Number_of_Claims', 'Total_Claim_Amount','Total_Payment_Amount','Total_Commission_Amount']

date_vars = ['Date_of_Appointment', 'Date_of_Termination', 'Agent_Date_of_Birth' ]

In [ ]:
# Convert date columns
date_vars = ['Date_of_Appointment','Date_of_Termination','Agent_Date_of_Birth' ]
for var in date_vars:
    df_work[var]=pd.to_datetime(df_work[var].astype(str), format='%Y-%m-%d')

In [ ]:
# Convert to 'category' object type
for var in cat_vars:
    df_work[var] = df_work[var].astype('category')

In [ ]:
# Create Dummy Variables for categorical variables
for var in cat_vars:
    cat_list = 'var'+'_'+var
    cat_list = pd.get_dummies(df_work[var], prefix=var)
    dummy_data = df_work.join(cat_list)
    df_work = dummy_data

In [ ]:
# Check data shape after dummy encoding
df_work.shape

In [ ]:
# Remove original columns for which dummy variables have been created
vars_remove = ['Commission Class','Pay Method','Pay Frequency','Agent Type','Agent Branch Code',
            'Agent Reporting Level','Agent Gender']

for var in vars_remove:
    df_work=df_work.drop(var, axis=1)

In [ ]:
# Normalize numerical columns
scaler = MinMaxScaler()
df_work['Number_of_Policies'] = scaler.fit_transform(df_work['Number_of_Policies'].values.reshape(-1,1))
df_work['Number_of_Claims'] = scaler.fit_transform(df_work['Number_of_Claims'].values.reshape(-1,1))
df_work['Total_Claim_Amount'] = scaler.fit_transform(df_work['Total_Claim_Amount'].values.reshape(-1,1))
df_work['Total_Payment_Amount'] = scaler.fit_transform(df_work['Total_Payment_Amount'].values.reshape(-1,1))
df_work['Total_Commission_Amount'] = scaler.fit_transform(df_work['Total_Commission_Amount'].values.reshape(-1,1))

### Feature Engineering

In [ ]:
# Get current date
current_time = datetime.datetime.now() 

In [ ]:
# Create a new column 'Age' on the basis of date of birth of agent
df_work['age'] = current_time - df_work['Agent_Date_of_Birth']

In [ ]:
# Create a new column named 'service period' on the basis of difference between Agent Date of Registration and Termination.
# In case date of Termination is null we will replace it with current date and compute

service_period=[]

for Date_of_Termination,Date_of_Appointment in zip(df_work.iloc[:,2], df_work.iloc[:,1]):
    if Date_of_Termination is pd.NaT:
        service_period.append(abs(current_time-Date_of_Appointment))
    else:
        service_period.append(abs(Date_of_Termination-Date_of_Appointment))


In [ ]:
# Append the new column to dataframe
df_work['Service_Period']=service_period

# Remove the orginal columns
df_work=df_work.drop('Date_of_Appointment', axis=1)
df_work=df_work.drop('Date_of_Termination', axis=1)
df_work=df_work.drop('Agent_Date_of_Birth', axis=1)

# Remove Agent Number since it is not critical for prediction
df_work=df_work.drop('Agent Number', axis=1)

In [ ]:
# Check contents once
df_work.head()

In [ ]:
# Normalize the new columns 
scaler = MinMaxScaler()
df_work['age'] = scaler.fit_transform(df_work['age'].values.reshape(-1,1))
df_work['Service_Period'] = scaler.fit_transform(df_work['Service_Period'].values.reshape(-1,1))

In [ ]:
# Create separate dataset for feature and target columns
X = df_work.drop('Churn', axis=1)
y = df_work['Churn']

### Performing SMOTE operation to handle class imbalance

In [ ]:
# Perform Test/Train Split
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.3,random_state=11)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
# To handle class imbalance performing SMOTE

from imblearn.over_sampling import SMOTE
SMO = SMOTE(random_state=11)

X_train_smo, y_train_smo = SMO.fit_sample(X_train, y_train)
columns = X_train.columns

X_train_smo = pd.DataFrame(data = X_train_smo, columns = columns )
y_train_smo = pd.DataFrame(data = y_train_smo, columns = ['Churn'])

# we can Check the numbers of our data
print("length of oversampled data is ",len(X_train_smo))
print("Number of Non Churned agents in oversampled data",len(y_train_smo[y_train_smo['Churn']== 0 ]))
print("Number of Churned agents in oversampled data",len(y_train_smo[y_train_smo['Churn']== 1 ]))
print("Proportion of Non Churned agents in oversampled data is ",len(y_train_smo[y_train_smo['Churn']== 0])/len(X_train_smo))
print("Proportion of Churned Agents in oversampled data is ",len(y_train_smo[y_train_smo['Churn']== 1 ])/len(X_train_smo))

# Model Application

### TPOT Model

In [ ]:
#Import tpot classifier from tpot and initialize it to run for max generation = 5 OR max time = 60 mins
from tpot import TPOTClassifier
tpot = TPOTClassifier(generations=5, verbosity=2, max_time_mins=60, n_jobs = -1)
tpot.fit(X_train_smo, y_train_smo)

In [ ]:
# Fit the pipeline
TPOT_Fit_pipeline=tpot.fitted_pipeline_
TPOT_Fit_pipeline

In [ ]:
# Print score
print(tpot.score(X_test, y_test))

In [ ]:
# Make Predictions
y_pred=TPOT_Fit_pipeline.predict(X_test)

In [ ]:
# Make Confusion Matrix to compare results against actuals
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix  

In [ ]:
# Get Accuracy Scores
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Precision:",metrics.precision_score(y_test, y_pred))
print("Recall:",metrics.recall_score(y_test, y_pred))

In [ ]:
# Print ROC AUC Curve

logit_roc_auc = roc_auc_score(y_test, TPOT_Fit_pipeline.predict(X_test))
fpr, tpr, thresholds = roc_curve(y_test, TPOT_Fit_pipeline.predict_proba(X_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='TPOT Classifier (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

### Run the model on entire dataset and save the output in a csv file

In [ ]:
# Copy the original data set
df_test = df_orig.copy()

In [ ]:
# Separate features into categorical, numerical and category
cat_vars = ['Commission Class','Pay Method','Pay Frequency','Agent Type','Agent Branch Code',
            'Agent Reporting Level','Agent Gender']

num_vars = ['Number_of_Policies', 'Number_of_Claims', 'Total_Claim_Amount','Total_Payment_Amount','Total_Commission_Amount']

date_vars = ['Date_of_Appointment', 'Date_of_Termination', 'Agent_Date_of_Birth' ]

In [ ]:
# Convert date columns
date_vars = ['Date_of_Appointment','Date_of_Termination','Agent_Date_of_Birth' ]
for var in date_vars:
    df_test[var]=pd.to_datetime(df_test[var].astype(str), format='%Y-%m-%d')

In [ ]:
# Convert to 'category' object type
for var in cat_vars:
    df_test[var] = df_test[var].astype('category')

In [ ]:
# Create Dummy Variables for categorical variables
for var in cat_vars:
    cat_list = 'var'+'_'+var
    cat_list = pd.get_dummies(df_test[var], prefix=var)
    dummy_data = df_test.join(cat_list)
    df_test = dummy_data

In [ ]:
# Check data shape after dummy encoding
df_test.shape

In [ ]:
# Remove original columns for which dummy variables have been created and also the taget value 'Churn'
vars_remove = ['Commission Class','Pay Method','Pay Frequency','Agent Type','Agent Branch Code',
            'Agent Reporting Level','Agent Gender', 'Churn']

for var in vars_remove:
    df_test=df_test.drop(var, axis=1)

### Feature Engineering

In [ ]:
# Get current date
current_time = datetime.datetime.now() 

In [ ]:
# Create a new column 'Age' on the basis of date of birth of agent
df_test['age'] = current_time - df_test['Agent_Date_of_Birth']

In [ ]:
# Create a new column named 'service period' on the basis of difference between Agent Date of Registration and Termination.
# In case date of Termination is null we will replace it with current date and compute

service_period=[]

for Date_of_Termination,Date_of_Appointment in zip(df_test.iloc[:,2], df_test.iloc[:,1]):
    if Date_of_Termination is pd.NaT:
        service_period.append(abs(current_time-Date_of_Appointment))
    else:
        service_period.append(abs(Date_of_Termination-Date_of_Appointment))


In [ ]:
# Append the new column to dataframe
df_test['Service_Period']=service_period

# Remove the orginal columns
df_test=df_test.drop('Date_of_Appointment', axis=1)
df_test=df_test.drop('Date_of_Termination', axis=1)
df_test=df_test.drop('Agent_Date_of_Birth', axis=1)

# Remove Agent Number since it is not critical for prediction
df_test=df_test.drop('Agent Number', axis=1)

In [ ]:
# Normalize the new columns 
scaler = MinMaxScaler()
df_test['age'] = scaler.fit_transform(df_test['age'].values.reshape(-1,1))
df_test['Service_Period'] = scaler.fit_transform(df_test['Service_Period'].values.reshape(-1,1))

In [ ]:
# Make predictions
prediction_class = TPOT_Fit_pipeline.predict(df_test)
prediction_prob = TPOT_Fit_pipeline.predict_proba(df_test)

# Get Individual Probabilities
not_churn, churn = (prediction_prob).T

In [ ]:
# Save original dataframe in a separate dataframe
df_results = df_orig.copy()

In [ ]:
# Remove target column
df_results=df_results.drop('Churn',axis=1)

In [ ]:
# Replace target column with predicted value of '0' or '1' along with respective probablities
df_results['notchurn'] = not_churn
df_results['churn'] = churn
df_results['churn_prediction_class'] = prediction_class

### Though care has been taken in testing the query output, still double check for duplicate rows for an agent in the csv file

In [ ]:
df_results.to_csv('EX_SOURCE_AGENT_CHURN_LIFE.csv', index = None, header=True) 

### Save Model as PMML file

In [ ]:
joblib.dump(TPOT_Fit_pipeline, 'life_agent_churn.pmml') 